In [1]:
import scanpy as sc
import pandas as pd

/opt/conda/envs/spatialdata/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/opt/conda/envs/spatialdata/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/opt/conda/envs/spatialdata/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/opt/conda/envs/spatialdata/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `anndata` is deprecated. Import anndata.io.read_mtx instead.
  warnings.warn(msg, FutureWarning)
/opt/conda/envs/spatialdata/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is de

In [3]:
def load_fry(frydir, which_counts=['S','A'], verbose=False):
    import json
    import os
    import pandas as pd
    
    fpath = os.path.sep.join([frydir, "quant.json"])
    if os.path.exists(fpath):
        fpath = os.path.sep.join([frydir, "meta_info.json"])

    meta_info = json.load(open(fpath))
    ng = meta_info['num_genes']
    usa_mode = meta_info['usa_mode']

    if usa_mode:
        assert(len(which_counts) > 0)
        if verbose:
            print("processing input in USA mode, will return {}".format("+".join(which_counts)))
    elif verbose:
        print("processing input in standard mode, will return spliced count")

    af_raw = scanpy.read_mtx(os.path.sep.join([frydir, "alevin", "quants_mat.mtx"]))
    ng = int(ng/3) if usa_mode else ng
    afg = [ l.rstrip() for l in open(os.path.sep.join([frydir, "alevin", "quants_mat_cols.txt"])).readlines()][:ng]
    afg_df =  pd.DataFrame(afg, columns=["gene_ids"])
    afg_df = afg_df.set_index("gene_ids")
    
    abc = [ l.rstrip() for l in open(os.path.sep.join([frydir, "alevin", "quants_mat_rows.txt"])).readlines() ]
    abc_df = pd.DataFrame(abc, columns=['barcodes'])
    abc_df.index = abc_df['barcodes']
    
    x = af_raw.X
    if usa_mode:
        rd = {'S' : range(0,ng), 'U' : range(ng, 2*ng), 'A' : range(2*ng,3*ng)}
        o = x[:, rd[which_counts[0]]]
        for wc in which_counts[1:]:
            o += x[:, rd[wc]]
    else:
        o = x
        
    af = scanpy.AnnData(o.T, var=abc_df, obs=afg_df)
    af = af.T
    return af

In [2]:
#pip install scvi-tools
#pip install pyroe
#from pyroe import load_fry

In [4]:
sc.settings.verbosity = 1
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80, frameon=False, figsize=(3, 3), facecolor="white")

-----
anndata     0.11.1
scanpy      1.9.3
-----
PIL                 9.5.0
asciitree           NA
asttokens           NA
cffi                1.17.1
cloudpickle         2.2.1
comm                0.2.2
cycler              0.10.0
cython_runtime      NA
dask                2023.5.1
dateutil            2.9.0
debugpy             1.8.6
decorator           5.1.1
defusedxml          0.7.1
entrypoints         0.4
exceptiongroup      1.2.2
executing           2.1.0
fasteners           0.18
h5py                3.8.0
igraph              0.10.4
ipykernel           6.26.0
ipython_genutils    0.2.0
jedi                0.19.1
jinja2              3.1.4
joblib              1.2.0
kiwisolver          1.4.4
leidenalg           0.9.1
llvmlite            0.43.0
markupsafe          2.1.5
matplotlib          3.7.1
mpl_toolkits        NA
msgpack             1.0.5
natsort             8.3.1
numba               0.60.0
numcodecs           0.11.0
numpy               1.26.4
packaging           24.1
pandas             

In [5]:

# Path to your Alevin output directory (which includes `quants_mat.gz`)
adata = load_fry("/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_multiscale_data_check/WS_SKN_KCL10525738_S1/")


FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_multiscale_data_check/WS_SKN_KCL10525738_S1//quant.json'